In [1]:
from fastapi import FastAPI

from fastapi.openapi.docs import get_swagger_ui_html
from fastapi.openapi.utils import get_openapi

import secrets

from fastapi import Depends, FastAPI, HTTPException, status
from fastapi.security import HTTPBasic, HTTPBasicCredentials

app = FastAPI(docs_url=None, redoc_url=None, openapi_url = None)

security = HTTPBasic()


def get_current_username(credentials: HTTPBasicCredentials = Depends(security)):
    correct_username = secrets.compare_digest(credentials.username, "user")
    correct_password = secrets.compare_digest(credentials.password, "password")
    if not (correct_username and correct_password):
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Incorrect email or password",
            headers={"WWW-Authenticate": "Basic"},
        )
    return credentials.username


@app.get("/docs")
async def get_documentation(username: str = Depends(get_current_username)):
    return get_swagger_ui_html(openapi_url="/openapi.json", title="docs")


@app.get("/openapi.json")
async def openapi(username: str = Depends(get_current_username)):
    return get_openapi(title = "FastAPI", version="0.1.0", routes=app.routes)


In [2]:
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8080)

INFO:     Started server process [5598]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)


INFO:     127.0.0.1:60627 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:60629 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:60629 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:60632 - "GET /openapi.json HTTP/1.1" 401 Unauthorized
INFO:     127.0.0.1:60632 - "GET /openapi.json HTTP/1.1" 401 Unauthorized
INFO:     127.0.0.1:60632 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:60632 - "GET /docs HTTP/1.1" 401 Unauthorized
INFO:     127.0.0.1:60635 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:60635 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:60635 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:60643 - "GET /hello HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5598]


In [3]:
app.openapi_schema